In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter


writer = SummaryWriter(log_dir="runs/cifar10_experiment")  # Skapar en mapp för loggarna
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

train_size = int(0.7 * len(full_trainset))
val_size = len(full_trainset) - train_size 

trainset, valset = random_split(full_trainset, [train_size, val_size])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Kontrollera storlekar
print(f"Training set: {len(trainset)}, Validation set: {len(valset)}, Test set: {len(testset)}")

import torch.nn as nn
import torch.nn.functional as F

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 9, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(9, 24, 3, padding = 1)
        self.fc1 = nn.Linear(24 * 7 * 7, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.tanh(self.conv1(x)))
        x = self.pool(torch.tanh(self.conv2(x)))
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)  # Ingen aktiveringsfunktion här, eftersom vi använder CrossEntropyLoss
        return x

net = Net()

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
epochs = 30

for epoch in range(epochs): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 200 == 0:  # Printa loss var 200:e batch
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            avg_loss = running_loss / 200
            writer.add_scalar('Training Loss', avg_loss, epoch * len(trainloader) + i)
            running_loss = 0.0

print('Finished Training')

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')
writer.add_scalar('Test Accuracy', accuracy, epochs)

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

total_correct = sum(correct_pred.values()) 
total_images = sum(total_pred.values())    
total_accuracy = 100 * total_correct / total_images
print(f'Total accuracy: {total_accuracy:.2f} %')
writer.add_scalar('Total Accuracy', total_accuracy, epochs)
writer.close()






Files already downloaded and verified
Files already downloaded and verified
Training set: 35000, Validation set: 15000, Test set: 10000
[1,   200] loss: 1.750
[1,   400] loss: 1.506
[2,   200] loss: 1.282
[2,   400] loss: 1.243
[3,   200] loss: 1.096
[3,   400] loss: 1.068
[4,   200] loss: 0.953
[4,   400] loss: 0.953
[5,   200] loss: 0.847
[5,   400] loss: 0.857
[6,   200] loss: 0.748
[6,   400] loss: 0.781
[7,   200] loss: 0.679
[7,   400] loss: 0.702
[8,   200] loss: 0.583
[8,   400] loss: 0.637
[9,   200] loss: 0.515
[9,   400] loss: 0.555
[10,   200] loss: 0.451
[10,   400] loss: 0.482
[11,   200] loss: 0.397
[11,   400] loss: 0.420
[12,   200] loss: 0.327
[12,   400] loss: 0.362
[13,   200] loss: 0.272
[13,   400] loss: 0.297
[14,   200] loss: 0.220
[14,   400] loss: 0.248
[15,   200] loss: 0.185
[15,   400] loss: 0.222
[16,   200] loss: 0.144
[16,   400] loss: 0.178
[17,   200] loss: 0.130
[17,   400] loss: 0.144
[18,   200] loss: 0.107
[18,   400] loss: 0.127
[19,   200] loss: 

NameError: name 'correct' is not defined